<a href="https://colab.research.google.com/github/fanzhh/Auto-deploy-sap-and-keepalive/blob/main/preprocess_gmp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import ai

response = ai.generate_text("What is the capital of France?")
print(response)

The capital of France is **Paris**.


In [2]:
try:
    from google.colab import ai
    COLAB_AVAILABLE = True
    print("✓ Google Colab AI已加载")
except ImportError:
    COLAB_AVAILABLE = False
    print("⚠ 未在Colab环境中，将使用备用配置")

✓ Google Colab AI已加载


In [3]:
import os
import json
import time
import logging
import re
from pathlib import Path
from typing import List, Dict, Optional
from dataclasses import dataclass
import threading
from datetime import datetime
import requests

# Colab环境配置
class ColabConfig:
    """Colab环境配置类"""

    def __init__(self):
        self.colab_available = COLAB_AVAILABLE
        # 使用当前目录而不是/content
        self.working_dir = os.getcwd()
        self.gmps_dir = f"{self.working_dir}/processed"  # 读取processed目录中的文件
        self.jsons_dir = f"{self.working_dir}/jsons"     # 输出到jsons目录
        self.processed_dir = f"{self.working_dir}/processed"  # processed目录

    def setup_directories(self):
        """创建必要的工作目录"""
        for dir_path in [self.jsons_dir]:
            if not os.path.exists(dir_path):
                os.makedirs(dir_path, exist_ok=True)
                print(f"✓ 创建目录: {dir_path}")

        # 检查processed目录
        if not os.path.exists(self.processed_dir):
            print(f"⚠️ 警告: processed目录不存在: {self.processed_dir}")
        else:
            print(f"✓ 找到processed目录: {self.processed_dir}")

        # 检查jsons目录
        if os.path.exists(self.jsons_dir):
            print(f"✓ 找到jsons目录: {self.jsons_dir}")

        print(f"✓ 当前工作目录: {self.working_dir}")

    def get_upload_instructions(self):
        """获取文件处理说明"""
        return f"""
文件处理说明：

1. 输入文件: 从 processed/ 目录读取.txt文件
2. 输出文件: 生成的JSON文件保存到 jsons/ 目录

当前目录结构：
{self.working_dir}/
├── processed/       ← 读取此目录中的.txt文件
├── jsons/          ← 输出的JSON文件将保存到这里
└── qa_generator_colab_part*.py ← 处理脚本
"""

# 配置日志
def setup_logging():
    """设置日志配置"""
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        force=True
    )
    return logging.getLogger(__name__)

# 全局变量
logger = setup_logging()
config = ColabConfig()

# 初始化环境
def initialize_environment():
    """初始化Colab环境"""
    print("=" * 50)
    print("GMP问答对生成器 - Colab版本")
    print("=" * 50)

    # 设置工作目录
    config.setup_directories()

    # 显示上传说明
    print(config.get_upload_instructions())

    # 检查Colab AI
    if config.colab_available:
        print("✓ 将使用Google Colab内置AI")
    else:
        print("⚠ 将使用备用API配置")

    print("\n环境初始化完成！")
    return True

# 使用说明
def show_usage_instructions():
    """显示使用说明"""
    instructions = """
使用步骤：

1. 📁 上传文件：将.txt文件上传到 /content/gmps/ 目录
2. 🚀 运行处理：执行第4部分的代码
3. 📊 查看结果：在 /content/jsons/ 目录中查看生成的JSON文件

命令示例：
# 处理单个文件
process_single_file("文件名.txt")

# 处理所有文件
process_all_files()

# 查看处理进度
show_progress()
"""
    print(instructions)

# 运行初始化
if __name__ == "__main__":
    initialize_environment()
    show_usage_instructions()

GMP问答对生成器 - Colab版本
✓ 找到processed目录: /content/processed
✓ 找到jsons目录: /content/jsons
✓ 当前工作目录: /content

文件处理说明：

1. 输入文件: 从 processed/ 目录读取.txt文件
2. 输出文件: 生成的JSON文件保存到 jsons/ 目录

当前目录结构：
/content/
├── processed/       ← 读取此目录中的.txt文件
├── jsons/          ← 输出的JSON文件将保存到这里
└── qa_generator_colab_part*.py ← 处理脚本

✓ 将使用Google Colab内置AI

环境初始化完成！

使用步骤：

1. 📁 上传文件：将.txt文件上传到 /content/gmps/ 目录
2. 🚀 运行处理：执行第4部分的代码
3. 📊 查看结果：在 /content/jsons/ 目录中查看生成的JSON文件

命令示例：
# 处理单个文件
process_single_file("文件名.txt")

# 处理所有文件
process_all_files()

# 查看处理进度
show_progress()



In [9]:
import os
import json
import time
import logging
import re
from pathlib import Path
from typing import List, Dict, Optional
from dataclasses import dataclass
import threading
from datetime import datetime
import requests
import asyncio
from concurrent.futures import Future

# 设置默认logger（如果part1没有设置）
logger = logging.getLogger(__name__)
if not logger.handlers:
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

@dataclass
class ProcessingStats:
    """处理统计信息"""
    total_files: int = 0
    processed_files: int = 0
    total_qa_pairs: int = 0
    total_chunks: int = 0
    errors: int = 0
    start_time: float = 0

    def get_progress(self) -> float:
        """获取处理进度百分比"""
        if self.total_files == 0:
            return 0.0
        return (self.processed_files / self.total_files) * 100

class ProgressDisplay:
    """进度显示类 - 单行刷新模式"""
    def __init__(self):
        self.stats = ProcessingStats()
        self.lock = threading.Lock()
        self.current_filename = ""
        self.last_update_time = 0

    def start(self, total_files: int):
        """开始处理"""
        with self.lock:
            self.stats.total_files = total_files
            self.stats.start_time = time.time()
            self.last_update_time = time.time()
            logger.info(f"开始处理 {total_files} 个文件...")
            print(f"🚀 开始处理 {total_files} 个文件...", end="")

    def update_file(self, filename: str, qa_count: int = 0):
        """更新文件处理状态（单行显示）"""
        with self.lock:
            self.stats.processed_files += 1
            self.stats.total_qa_pairs += qa_count
            progress = self.stats.get_progress()
            self.current_filename = filename

            # 计算处理速度
            total_time = time.time() - self.stats.start_time
            if total_time > 0:
                speed = self.stats.processed_files / total_time
                eta = (self.stats.total_files - self.stats.processed_files) / speed if speed > 0 else 0
            else:
                speed = 0
                eta = 0

            # 构建进度条
            bar_length = 20
            filled_length = int(bar_length * self.stats.processed_files // self.stats.total_files)
            bar = '█' * filled_length + '-' * (bar_length - filled_length)

            # 单行刷新进度
            progress_line = (
                f"\r📊 进度: [{bar}] {progress:.1f}% "
                f"({self.stats.processed_files}/{self.stats.total_files}) "
                f"Q&A: {self.stats.total_qa_pairs} "
                f"速度: {speed:.2f}文件/s "
                f"ETA: {eta:.0f}s "
                f"当前: {filename}"
            )

            print(progress_line, end="", flush=True)
            self.last_update_time = time.time()

    def update_chunk_progress(self, current_chunk: int, total_chunks: int, filename: str):
        """更新文本块处理进度"""
        if total_chunks <= 0:
            return

        # 限制更新频率，避免刷屏
        current_time = time.time()
        if current_time - self.last_update_time < 0.5:  # 0.5秒更新一次
            return

        progress = (current_chunk / total_chunks) * 100
        file_progress = ((self.stats.processed_files) / self.stats.total_files) * 100

        # 构建进度条
        bar_length = 15
        filled_length = int(bar_length * current_chunk // total_chunks)
        chunk_bar = '▓' * filled_length + '░' * (bar_length - filled_length)

        # 单行刷新文本块进度
        progress_line = (
            f"\r📝 {filename} [{chunk_bar}] {progress:.0f}% "
            f"({current_chunk}/{total_chunks}块) "
            f"总进度: {file_progress:.1f}% "
            f"Q&A: {self.stats.total_qa_pairs}"
        )

        print(progress_line, end="", flush=True)
        self.last_update_time = current_time

    def add_error(self):
        """增加错误计数"""
        with self.lock:
            self.stats.errors += 1

    def update_chunks(self, chunk_count: int):
        """更新处理的文本块数量"""
        with self.lock:
            self.stats.total_chunks += chunk_count

    def finish_file(self, filename: str, qa_count: int = 0):
        """完成文件处理"""
        with self.lock:
            self.stats.total_qa_pairs += qa_count
            # 刷新最终状态
            self.update_file(f"✅ {filename}", qa_count)

    def show_summary(self):
        """显示处理总结"""
        total_time = time.time() - self.stats.start_time
        avg_speed = self.stats.processed_files / total_time if total_time > 0 else 0

        print("\n" + "=" * 60)
        print("🎉 处理完成总结:")
        print("-" * 60)
        print(f"📁 总文件数: {self.stats.total_files}")
        print(f"✅ 成功处理: {self.stats.processed_files}")
        print(f"❌ 处理失败: {self.stats.errors}")
        print(f"💬 总Q&A对数: {self.stats.total_qa_pairs:,}")
        print(f"📄 总文本块数: {self.stats.total_chunks}")
        print(f"⏱️  总耗时: {total_time:.2f}秒")
        print(f"⚡ 平均速度: {avg_speed:.2f}文件/秒")

        if self.stats.processed_files > 0:
            avg_qa_per_file = self.stats.total_qa_pairs / self.stats.processed_files
            avg_qa_per_chunk = self.stats.total_qa_pairs / self.stats.total_chunks if self.stats.total_chunks > 0 else 0
            print(f"📊 平均Q&A/文件: {avg_qa_per_file:.1f}")
            print(f"📊 平均Q&A/文本块: {avg_qa_per_chunk:.1f}")

        print("-" * 60)

        # 显示输出目录
        try:
            from part1 import config
            print(f"📂 结果文件保存在: {config.jsons_dir}/")
        except:
            print(f"📂 结果文件保存在: ./jsons/")

        print("=" * 60)

# Helper to run async functions in a new thread with a new event loop
def _run_async_in_new_thread(coro, result_future):
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    try:
        result = loop.run_until_complete(coro)
        result_future.set_result(result)
    except Exception as e:
        result_future.set_exception(e)
    finally:
        loop.close()

class ColabAIClient:
    """Google Colab AI客户端"""

    def __init__(self):
        self.colab_available = False
        self.api_client = None
        try:
            from google.colab import ai
            self.colab_available = True
            logger.info("✓ Google Colab AI客户端已初始化")
        except ImportError:
            logger.warning("⚠ 未检测到Colab环境，将使用备用客户端")
            self._init_fallback_client()

    def _init_fallback_client(self):
        """初始化备用API客户端"""
        try:
            # 如果提供了备用API配置
            API_BASE_URL = os.getenv("API_BASE_URL", "http://localhost:3033")
            API_TOKEN = os.getenv("API_TOKEN", "")
            if API_TOKEN:
                self.api_client = type('APIClient', (), {
                    'base_url': API_BASE_URL,
                    'api_token': API_TOKEN
                })()
                logger.info("✓ 备用API客户端已初始化")
            else:
                logger.warning("⚠ 未配置备用API，将使用模拟模式")
                self.api_client = None
        except Exception as e:
            logger.error(f"备用API初始化失败: {e}")

    async def generate_qa_pairs_async(self, prompt: str) -> Dict:
        """异步生成问答对"""
        if self.colab_available:
            try:
                from google.colab import ai
                # Removed 'await' as ai.generate_text is a synchronous function
                response_content = ai.generate_text(prompt)
                return {
                    "content": response_content, # Changed from response.text to response_content
                    "model": "google-colab-ai",
                    "success": True
                }
            except Exception as e:
                logger.error(f"Colab AI调用失败: {e}")
                return await self._fallback_generation(prompt)
        else:
            return await self._fallback_generation(prompt)

    async def _fallback_generation(self, prompt: str) -> Dict:
        """备用生成方法"""
        if self.api_client:
            try:
                # 模拟API调用
                return {
                    "content": self._simulate_ai_response(prompt),
                    "model": "fallback-simulation",
                    "success": True
                }
            except Exception as e:
                logger.error(f"备用API调用失败: {e}")
                return {"content": "", "model": "none", "success": False}
        else:
            # 模拟模式
            logger.warning("使用模拟模式生成问答对")
            return {
                "content": self._simulate_ai_response(prompt),
                "model": "simulation",
                "success": True
            }

    def _simulate_ai_response(self, prompt: str) -> str:
        """模拟AI响应（仅用于演示）"""
        return '''Q: 什么是GMP？
A: GMP（Good Manufacturing Practice）是药品生产质量管理规范的简称，是一套用于确保药品质量的法规和标准。

Q: GMP的基本要求是什么？
A: GMP的基本要求包括：人员资质、厂房设施、设备维护、物料管理、生产过程控制、质量检验、文件记录、质量体系等方面。

Q: 药品生产企业需要遵守哪些GMP法规？
A: 药品生产企业需要遵守国家药品监督管理局发布的《药品生产质量管理规范》以及相关的技术指南和要求。'''

    def generate_qa_pairs(self, prompt: str) -> Dict:
        """同步生成问答对（兼容性方法），通过新线程避免事件循环冲突"""
        result_future = Future()
        thread = threading.Thread(target=_run_async_in_new_thread, args=(self.generate_qa_pairs_async(prompt), result_future))
        thread.start()
        thread.join() # Ensure the thread completes before returning
        return result_future.result() # Blocks until the async call is complete

class QAGeneratorColab:
    """Colab版本的问答对生成器"""

    def __init__(self):
        self.ai_client = ColabAIClient()
        self.progress = ProgressDisplay()

        # 确保输出目录存在
        # 配置将在初始化时设置
        self.config = None
        os.makedirs("/content/jsons", exist_ok=True)

    def generate_qa_pairs(self, text: str) -> List[Dict]:
        """生成问答对"""
        if len(text.strip()) < 50:
            logger.warning("文本过短，跳过处理")
            return []

        # 构建提示词
        prompt = f"""你是一个专业的文本分析助手，擅长从技术文档中提取关键信息并组织成清晰的问答对。

【任务】
请仔细阅读以下文本，并从中生成一系列有意义的问答对。问答对应覆盖文本中的核心概念、关键流程、监管要求和重要结论。

【生成要求】
1. **问题**：应基于文本内容提出，问题明确、具体，能抓住一个独立的知识点。
2. **答案**：必须直接从文本中提炼和总结，确保信息准确、完整。
3. **数量与质量**：生成2-10个高质量的问答对，确保覆盖文本的不同部分。
4. **格式**：请严格按照以下格式输出：

Q: [这里写问题]
A: [这里写答案]

【文本】
{text}

现在，请开始生成问答对："""

        # Suppress logging during AI call to avoid breaking progress bar
        original_level = logging.getLogger().level
        logging.getLogger().setLevel(logging.ERROR) # Temporarily set to ERROR to suppress INFO/WARNING

        try:
            # 调用AI
            result = self.ai_client.generate_qa_pairs(prompt)

            if not result["success"]:
                logger.error(f"AI调用失败: {result}")
                return []

            # 提取回复内容
            response_text = result["content"]
            # logger.info(f"使用模型: {result['model']}") # Suppress this log during progress update

            # 提取问答对
            qa_pairs = self._extract_qa_pairs_from_response(response_text)

            if qa_pairs:
                # logger.info(f"成功生成 {len(qa_pairs)} 个问答对") # Suppress this log during progress update
                pass
            else:
                # logger.warning("未能从响应中提取有效的问答对") # Suppress this log during progress update
                pass
            return qa_pairs

        except Exception as e:
            logger.error(f"生成问答对时出错: {str(e)}")
            return []
        finally:
            logging.getLogger().setLevel(original_level) # Restore original logging level

    def _extract_qa_pairs_from_response(self, response_text: str) -> List[Dict]:
        """从响应中提取问答对"""
        try:
            # 使用正则表达式匹配 Q: ... A: ... 格式
            qa_pairs = []
            qa_pattern = r'Q:\s*(.+?)\s*A:\s*([^Q]+?)(?=\s*Q:|$)'
            matches = re.findall(qa_pattern, response_text, re.DOTALL)

            for match in matches:
                question = match[0].strip()
                answer = match[1].strip()

                if question and answer and len(answer) > 10:
                    qa_pairs.append({
                        "instruction": question,
                        "input": "",
                        "output": answer
                    })
                    # logger.info(f"提取Q&A对: {question[:50]}...") # Suppress this log during progress update

            if qa_pairs:
                # logger.info(f"成功解析 {len(qa_pairs)} 个问答对") # Suppress this log during progress update
                pass
            else:
                # logger.debug("未找到有效的Q:A格式问答对") # Suppress this log during progress update
                pass
            return qa_pairs

        except Exception as e:
            logger.error(f"提取问答对时出错: {str(e)}")
            return []

    def save_qa_pairs(self, filename: str, qa_pairs: List[Dict]) -> bool:
        """保存问答对到JSON文件"""
        output_path = Path("/content/jsons") / f"{filename}.json"

        try:
            with open(output_path, 'w', encoding='utf-8') as f:
                json.dump(qa_pairs, f, ensure_ascii=False, indent=2)

            # logger.info(f"成功保存 {len(qa_pairs)} 个问答对到 {output_path}") # Suppress this log during progress update
            return True

        except Exception as e:
            logger.error(f"保存问答对时出错: {str(e)}")
            return False

    def append_qa_pairs(self, filename: str, qa_pairs: List[Dict]) -> bool:
        """追加问答对到JSON文件"""
        output_path = Path("/content/jsons") / f"{filename}.json"
        temp_path = output_path.with_suffix('.tmp')

        try:
            # 读取现有数据
            existing_data = []
            if output_path.exists():
                with open(output_path, 'r', encoding='utf-8') as f:
                    existing_data = json.load(f)

            # 追加新数据
            existing_data.extend(qa_pairs)

            # 写入临时文件
            with open(temp_path, 'w', encoding='utf-8') as f:
                json.dump(existing_data, f, ensure_ascii=False, indent=2)

            # 重命名临时文件
            temp_path.replace(output_path)

            # logger.info(f"增量保存成功: {len(qa_pairs)} 个问答对已追加到 {output_path.name}") # Suppress this log during progress update
            # logger.info(f"当前总问答对数: {len(existing_data)}") # Suppress this log during progress update
            return True

        except Exception as e:
            logger.error(f"保存问答对时出错: {str(e)}")
            # 清理临时文件
            if temp_path.exists():
                temp_path.unlink()
            return False

# 导出主要类
__all__ = ['QAGeneratorColab', 'ProcessingStats', 'ProgressDisplay', 'ColabAIClient']

In [5]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
GMP问答对生成模块 - Colab版本
第3部分：文本处理和分块功能
"""

import os
import re
from pathlib import Path
from typing import List, Dict, Optional
import logging

# 设置默认logger（如果part1没有设置）
logger = logging.getLogger(__name__)
if not logger.handlers:
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class TextProcessor:
    """文本处理器 - 专门处理中文GMP文档"""

    def __init__(self):
        self.chinese_punctuation = ['。', '！', '？', '；', '：', '"', '"', ''', ''', '（', '）', '【', '】']
        self.line_endings = ['\n', '\r', '\r\n']

    def clean_text(self, text: str) -> str:
        """清理文本，去除OCR错误和格式问题"""
        if not text:
            return ""

        # 去除空白字符但保留中文格式
        text = re.sub(r'[ \t]+', ' ', text)  # 制表符和多个空格转为单个空格
        text = re.sub(r'\n{3,}', '\n\n', text)  # 多个换行转为两个
        text = re.sub(r' +\n', '\n', text)  # 行尾空格去除
        text = re.sub(r'\n +', '\n', text)  # 行首空格去除

        # 修复常见的OCR错误
        ocr_corrections = {
            '□': '',  # 方框字符
            '※': '',  # 参考标记
            '•': '',  # 项目符号
            '…': '...',  # 省略号统一
            '"': '"',  # 引号统一
            '"': '"',
            ''': "'",
            ''': "'"
        }

        for wrong, correct in ocr_corrections.items():
            text = text.replace(wrong, correct)

        # 去除控制字符但保留中文标点
        text = ''.join(char for char in text if ord(char) >= 32 or char in self.chinese_punctuation)

        # 修复数字和单位之间的空格
        text = re.sub(r'(\d+)\s+([个只片粒毫升克])(?![a-zA-Z])', r'\1\2', text)

        # 修复常见的GMP术语格式
        text = self._fix_gmp_terms(text)

        return text.strip()

    def _fix_gmp_terms(self, text: str) -> str:
        """修复GMP相关术语的格式"""
        # 修复条款编号格式
        text = re.sub(r'第\s*(\d+)\s*条', r'第\1条', text)
        text = re.sub(r'(\d+)\s*\.\s*(\d+)', r'\1.\2', text)  # 如 1. 2 -> 1.2

        # 修复常见的GMP缩写
        gmp_terms = {
            'G M P': 'GMP',
            'g m p': 'GMP',
            'H A C C P': 'HACCP',
            'I S O': 'ISO',
            'Q A': 'QA',
            'Q C': 'QC'
        }

        for term, correct in gmp_terms.items():
            text = re.sub(r'\b' + re.escape(term) + r'\b', correct, text, flags=re.IGNORECASE)

        return text

    def process_text_chunk(self, text: str, chunk_size: int = 800, overlap_size: int = 200) -> List[str]:
        """将文本分块处理，优化中文文本的分割"""
        if not text or len(text) <= chunk_size:
            return [text] if text.strip() else []

        chunks = []
        start = 0

        while start < len(text):
            # 计算块的结束位置
            end = start + chunk_size

            # 如果到达文本末尾，直接添加剩余部分
            if end >= len(text):
                chunks.append(text[start:].strip())
                break

            # 寻找最佳的分割点
            chunk = text[start:end]

            # 优先在句号、问号、感叹号处分割
            split_pos = self._find_best_split_point(chunk)

            if split_pos != -1:
                chunks.append(chunk[:split_pos + 1].strip())
                start = start + split_pos + 1 - overlap_size
            else:
                # 如果找不到合适的分割点，强制分割
                chunks.append(chunk.strip())
                start = end - overlap_size

        # 过滤空块和过短的块
        chunks = [chunk for chunk in chunks if chunk and len(chunk.strip()) > 50]

        logger.info(f"文本分块完成，共 {len(chunks)} 个块，平均长度: {sum(len(c) for c in chunks) / len(chunks):.0f} 字符")
        return chunks

    def _find_best_split_point(self, chunk: str) -> int:
        """寻找最佳的分割点"""
        # 分割点优先级：句号 > 问号 > 感叹号 > 分号 > 冒号 > 逗号 > 换行

        split_chars = ['。', '！', '？', '；', '：', '，', '\n']

        for char in split_chars:
            # 从后向前查找分割点
            for i in range(len(chunk) - 1, max(0, len(chunk) - 200), -1):
                if chunk[i] == char:
                    return i

        return -1

    def extract_sentences(self, text: str) -> List[str]:
        """提取句子，用于更细粒度的处理"""
        if not text:
            return []

        # 使用正则表达式分割句子
        sentences = re.split(r'[。！？；]', text)

        # 清理和过滤句子
        sentences = [s.strip() for s in sentences if s.strip() and len(s.strip()) > 10]

        return sentences

    def detect_document_type(self, text: str) -> str:
        """检测文档类型"""
        text_lower = text.lower()

        if any(keyword in text_lower for keyword in ['gmp', 'good manufacturing practice', '药品生产质量管理']):
            return 'GMP'
        elif any(keyword in text_lower for keyword in ['sop', '标准操作规程', '操作指南']):
            return 'SOP'
        elif any(keyword in text_lower for keyword in ['验证', 'validation', '确认']):
            return 'Validation'
        elif any(keyword in text_lower for keyword in ['培训', 'training', '培训记录']):
            return 'Training'
        elif any(keyword in text_lower for keyword in ['检查', 'inspection', '审计', 'audit']):
            return 'Inspection'
        else:
            return 'General'

    def extract_key_info(self, text: str) -> Dict[str, List[str]]:
        """提取关键信息"""
        key_info = {
            'requirements': [],
            'procedures': [],
            'responsibilities': [],
            'definitions': []
        }

        # 提取要求（通常包含"应"、"必须"、"不得"等关键词）
        requirement_patterns = [
            r'[^。！？]*?应[^。！？]*[。！？]',
            r'[^。！？]*?必须[^。！？]*[。！？]',
            r'[^。！？]*?不得[^。！？]*[。！？]',
            r'[^。！？]*?严禁[^。！？]*[。！？]'
        ]

        for pattern in requirement_patterns:
            matches = re.findall(pattern, text)
            key_info['requirements'].extend(matches)

        # 提取程序（通常包含"程序"、"步骤"、"方法"等）
        procedure_patterns = [
            r'[^。！？]*?程序[^。！？]*[。！？]',
            r'[^。！？]*?步骤[^。！？]*[。！？]',
            r'[^。！？]*?方法[^。！？]*[。！？]'
        ]

        for pattern in procedure_patterns:
            matches = re.findall(pattern, text)
            key_info['procedures'].extend(matches)

        # 提取职责（通常包含"负责"、"职责"等）
        responsibility_patterns = [
            r'[^。！？]*?负责[^。！？]*[。！？]',
            r'[^。！？]*?职责[^。！？]*[。！？]',
            r'[^。！？]*?权限[^。！？]*[。！？]'
        ]

        for pattern in responsibility_patterns:
            matches = re.findall(pattern, text)
            key_info['responsibilities'].extend(matches)

        # 提取定义（通常包含"定义"、"是指"等）
        definition_patterns = [
            r'[^。！？]*?定义[^。！？]*[。！？]',
            r'[^。！？]*?是指[^。！？]*[。！？]',
            r'[^。！？]*?为[^。！？]*[。！？]'
        ]

        for pattern in definition_patterns:
            matches = re.findall(pattern, text)
            key_info['definitions'].extend(matches)

        # 去重并清理
        for key in key_info:
            key_info[key] = list(set([item.strip() for item in key_info[key] if item.strip()]))

        return key_info

    def validate_text_quality(self, text: str) -> Dict[str, any]:
        """验证文本质量"""
        stats = {
            'total_chars': len(text),
            'chinese_chars': len(re.findall(r'[\u4e00-\u9fff]', text)),
            'english_chars': len(re.findall(r'[a-zA-Z]', text)),
            'numbers': len(re.findall(r'\d', text)),
            'punctuation': len(re.findall(r'[，。！？；：""''（）【】]', text)),
            'sentences': len(re.findall(r'[。！？]', text)),
            'quality_score': 0
        }

        # 计算质量评分
        if stats['total_chars'] > 0:
            chinese_ratio = stats['chinese_chars'] / stats['total_chars']
            sentence_ratio = stats['sentences'] / max(stats['total_chars'] / 50, 1)  # 假设每50字符一个句子

            # 质量评分（0-100）
            quality_score = 0
            if chinese_ratio > 0.7:  # 中文占比较高
                quality_score += 40
            if sentence_ratio > 0.5:  # 句子数量合理
                quality_score += 30
            if stats['punctuation'] > 0:  # 有标点符号
                quality_score += 20
            if stats['total_chars'] > 100:  # 文本长度足够
                quality_score += 10

            stats['quality_score'] = min(quality_score, 100)

        return stats

# 文件处理工具类
class FileManager:
    """文件管理器"""

    def __init__(self, config=None):
        self.config = config
        # 使用当前目录结构
        self.jsons_dir = "./jsons"
        self.processed_dir = "./processed"
        self.text_processor = TextProcessor()

    def load_text_file(self, file_path: Path) -> str:
        """加载文本文件"""
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                text = f.read()

            # 清理文本
            cleaned_text = self.text_processor.clean_text(text)

            # 验证文本质量
            quality_stats = self.text_processor.validate_text_quality(cleaned_text)
            logger.info(f"文本质量评分: {quality_stats['quality_score']}/100")

            if quality_stats['quality_score'] < 30:
                logger.warning("文本质量较低，可能影响问答对生成质量")

            return cleaned_text

        except Exception as e:
            logger.error(f"加载文件失败 {file_path}: {str(e)}")
            return ""

    def save_processed_text(self, original_path: Path, text: str) -> Path:
        """保存处理后的文本"""
        processed_path = Path(self.processed_dir) / f"{original_path.stem}_cleaned.txt"

        try:
            with open(processed_path, 'w', encoding='utf-8') as f:
                f.write(text)

            logger.info(f"处理后的文本已保存: {processed_path}")
            return processed_path

        except Exception as e:
            logger.error(f"保存处理文本失败: {str(e)}")
            return None

    def get_file_list(self, directory: str = None) -> List[Path]:
        """获取文件列表"""
        if directory is None:
            directory = self.processed_dir  # 默认从processed目录读取

        dir_path = Path(directory)
        if not dir_path.exists():
            logger.error(f"目录不存在: {directory}")
            return []

        files = list(dir_path.glob("*.txt"))
        logger.info(f"在 {directory} 中找到 {len(files)} 个文本文件")
        return files

    def check_existing_output(self, filename: str) -> bool:
        """检查是否已有输出文件"""
        output_path = Path(self.jsons_dir) / f"{filename}.json"
        return output_path.exists()

# 导出类
__all__ = ['TextProcessor', 'FileManager']

In [6]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
GMP问答对生成模块 - Colab版本
第4部分：主要执行逻辑和示例用法
"""

import os
import time
import json
from pathlib import Path
from typing import List, Dict
import logging

# 设置默认logger（如果part1没有设置）
logger = logging.getLogger(__name__)
if not logger.handlers:
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class GMPProcessor:
    """GMP文档处理器 - 主要执行逻辑"""

    def __init__(self):
        # from part2 import QAGeneratorColab
        # from part3 import FileManager, TextProcessor

        # 不依赖config，使用当前目录结构
        self.processed_dir = "./processed"
        self.jsons_dir = "./jsons"

        # 由于类已经在exec时导入到全局，直接使用
        global QAGeneratorColab, FileManager, TextProcessor
        self.qa_generator = QAGeneratorColab()
        self.file_manager = FileManager()  # 不传递config
        self.text_processor = TextProcessor()
        self.progress = self.qa_generator.progress

    def process_single_file(self, file_path: Path, chunk_size: int = 800, overlap_size: int = 200) -> bool:
        """处理单个文件"""
        try:
            logger.info(f"开始处理文件: {file_path.name}")

            # 加载和清理文本
            raw_text = self.file_manager.load_text_file(file_path)
            if not raw_text:
                logger.error(f"无法加载文件: {file_path}")
                return False

            # 检测文档类型
            doc_type = self.text_processor.detect_document_type(raw_text)
            logger.info(f"文档类型: {doc_type}")

            # 保存清理后的文本
            self.file_manager.save_processed_text(file_path, raw_text)

            # 文本分块
            chunks = self.text_processor.process_text_chunk(raw_text, chunk_size, overlap_size)
            if not chunks:
                logger.warning("文本分块后为空")
                return False

            self.progress.update_chunks(len(chunks))

            # 处理每个文本块
            total_qa_pairs = []
            filename = file_path.stem

            for i, chunk in enumerate(chunks):
                # 显示文本块处理进度
                self.progress.update_chunk_progress(i+1, len(chunks), file_path.name)

                # 生成问答对
                qa_pairs = self.qa_generator.generate_qa_pairs(chunk)

                if qa_pairs:
                    total_qa_pairs.extend(qa_pairs)

                    # 增量保存
                    self.qa_generator.append_qa_pairs(filename, qa_pairs)

                # 避免请求过于频繁
                time.sleep(1)

            # 完成文件处理
            self.progress.finish_file(file_path.name, len(total_qa_pairs))

            # 最终统计
            logger.info(f"文件 {file_path.name} 处理完成")
            logger.info(f"生成问答对总数: {len(total_qa_pairs)}")

            # self.progress.update_file(file_path.name, len(total_qa_pairs))  # 已在上面调用finish_file
            return True

        except Exception as e:
            logger.error(f"处理文件 {file_path} 时出错: {str(e)}")
            self.progress.add_error()
            return False

    def process_all_files(self, chunk_size: int = 800, overlap_size: int = 200, resume: bool = True) -> Dict:
        """处理所有文件"""
        # 获取文件列表
        files = self.file_manager.get_file_list()
        if not files:
            logger.error("未找到任何文本文件")
            return {"success": False, "message": "未找到文件"}

        # 过滤已处理的文件（如果启用续传）
        if resume:
            unprocessed_files = []
            for file_path in files:
                if not self.file_manager.check_existing_output(file_path.stem):
                    unprocessed_files.append(file_path)
                else:
                    logger.info(f"跳过已处理文件: {file_path.name}")

            files = unprocessed_files

        if not files:
            logger.info("所有文件都已处理完成")
            return {"success": True, "message": "所有文件已处理"}

        # 开始处理
        self.progress.start(len(files))
        success_count = 0
        failed_files = []

        for file_path in files:
            if self.process_single_file(file_path, chunk_size, overlap_size):
                success_count += 1
            else:
                failed_files.append(file_path.name)

        # 显示处理总结
        self.progress.show_summary()

        return {
            "success": True,
            "processed": success_count,
            "failed": len(failed_files),
            "failed_files": failed_files,
            "total_qa_pairs": self.progress.stats.total_qa_pairs
        }

    def preview_file(self, file_path: Path, preview_chars: int = 500) -> Dict:
        """预览文件内容"""
        try:
            text = self.file_manager.load_text_file(file_path)
            if not text:
                return {"success": False, "message": "无法加载文件"}

            # 清理文本
            cleaned_text = self.text_processor.clean_text(text)

            # 获取统计信息
            quality_stats = self.text_processor.validate_text_quality(cleaned_text)
            doc_type = self.text_processor.detect_document_type(cleaned_text)

            preview = cleaned_text[:preview_chars]
            if len(cleaned_text) > preview_chars:
                preview += "..."

            return {
                "success": True,
                "filename": file_path.name,
                "doc_type": doc_type,
                "total_length": len(cleaned_text),
                "quality_score": quality_stats['quality_score'],
                "preview": preview
            }

        except Exception as e:
            return {"success": False, "message": str(e)}

# 便捷函数
def list_available_files():
    """列出可用的文件"""
    # from part3 import FileManager

    global FileManager
    file_manager = FileManager()
    files = file_manager.get_file_list()

    if not files:
        print("📁 processed目录中未找到任何文本文件")
        print("请确保已将.txt文件放入processed目录")
        return []

    print(f"📁 在processed目录中找到 {len(files)} 个文本文件:")
    total_size = 0
    for i, file_path in enumerate(files, 1):
        file_size = file_path.stat().st_size if file_path.exists() else 0
        total_size += file_size
        print(f"  {i:2d}. {file_path.name:<30} ({file_size:,} bytes)")

    print(f"📊 总计: {len(files)} 个文件, {total_size:,} bytes")
    return files

def process_single_file(filename: str, chunk_size: int = 800):
    """处理单个文件的便捷函数"""
    if not filename.endswith('.txt'):
        filename += '.txt'

    file_path = Path("./processed") / filename
    if not file_path.exists():
        print(f"❌ 文件不存在: {file_path}")
        return False

    print(f"🚀 开始处理文件: {filename}")
    global GMPProcessor
    processor = GMPProcessor()
    return processor.process_single_file(file_path, chunk_size)

def process_all_files(chunk_size: int = 800, resume: bool = True):
    """处理所有文件的便捷函数"""
    global GMPProcessor
    processor = GMPProcessor()
    return processor.process_all_files(chunk_size=chunk_size, resume=resume)

def preview_file(filename: str):
    """预览文件的便捷函数"""
    if not filename.endswith('.txt'):
        filename += '.txt'

    file_path = Path("./processed") / filename
    if not file_path.exists():
        print(f"❌ 文件不存在: {file_path}")
        return None

    print(f"🔍 预览文件: {filename}")
    global GMPProcessor
    processor = GMPProcessor()
    return processor.preview_file(file_path)

def show_results():
    """显示处理结果"""
    json_dir = Path("./jsons")
    if not json_dir.exists():
        print("❌ jsons目录不存在")
        print("请确保已创建jsons目录")
        return

    json_files = list(json_dir.glob("*.json"))
    if not json_files:
        print("📂 jsons目录中未找到处理结果")
        print("请先运行处理函数生成问答对")
        return

    total_qa_pairs = 0
    total_size = 0

    print(f"\n📊 处理结果 ({len(json_files)} 个JSON文件):")
    print("=" * 60)

    for json_file in sorted(json_files):
        try:
            with open(json_file, 'r', encoding='utf-8') as f:
                qa_pairs = json.load(f)
                qa_count = len(qa_pairs)
                total_qa_pairs += qa_count

            file_size = json_file.stat().st_size
            total_size += file_size

            print(f"  📄 {json_file.stem:<30} {qa_count:4d} Q&A  ({file_size:,} bytes)")

        except Exception as e:
            print(f"  ❌ {json_file.stem}: 读取错误 - {e}")

    print("-" * 60)
    print(f"📈 总计: {len(json_files)} 个文件, {total_qa_pairs:,} 个问答对, {total_size:,} bytes")

    # 显示平均统计
    if json_files:
        avg_qa = total_qa_pairs / len(json_files)
        print(f"📊 平均: {avg_qa:.1f} 问答对/文件")

    print(f"📂 结果保存在: {json_dir}/")
    print("=" * 60)

def download_results():
    """下载结果文件（Colab中使用）"""
    try:
        from google.colab import files

        json_dir = Path("./jsons")
        json_files = list(json_dir.glob("*.json"))

        if not json_files:
            print("❌ 未找到结果文件")
            print("请先运行处理函数生成问答对")
            return

        print(f"📦 准备下载 {len(json_files)} 个结果文件...")
        for i, json_file in enumerate(json_files, 1):
            print(f"  {i}/{len(json_files)}: {json_file.name}")
            files.download(str(json_file))

        print("✅ 下载完成！")

    except ImportError:
        print("⚠️ 此功能仅在Google Colab中可用")
        print("在本地环境中，请直接访问 ./jsons/ 目录")
    except Exception as e:
        print(f"❌ 下载失败: {e}")

# 使用示例和说明
def show_examples():
    """显示使用示例"""
    examples = """
使用示例:

# 1. 列出可用文件
files = list_available_files()

# 2. 预览文件
preview = preview_file("示例文件名.txt")
print(preview)

# 3. 处理单个文件
success = process_single_file("示例文件名.txt")

# 4. 处理所有文件
result = process_all_files(chunk_size=800, resume=True)

# 5. 查看结果
show_results()

# 6. 下载结果（仅在Colab中）
download_results()
"""
    print(examples)

# 主函数（用于直接运行）
def main():
    """主函数 - 交互式处理"""
    print("=" * 60)
    print("GMP问答对生成器 - Colab交互模式")
    print("=" * 60)

    # 显示可用文件
    files = list_available_files()
    if not files:
        return

    print("\n请选择操作:")
    print("1. 预览文件")
    print("2. 处理单个文件")
    print("3. 处理所有文件")
    print("4. 查看结果")
    print("5. 显示使用示例")

    try:
        choice = input("\n请输入选择 (1-5): ").strip()

        if choice == "1":
            filename = input("请输入文件名: ").strip()
            preview = preview_file(filename)
            if preview and preview.get("success"):
                print(f"\n文档类型: {preview['doc_type']}")
                print(f"质量评分: {preview['quality_score']}/100")
                print(f"预览内容:\n{preview['preview']}")

        elif choice == "2":
            filename = input("请输入文件名: ").strip()
            success = process_single_file(filename)
            print("处理完成" if success else "处理失败")

        elif choice == "3":
            chunk_size = input("请输入文本块大小 (默认800): ").strip()
            chunk_size = int(chunk_size) if chunk_size.isdigit() else 800
            result = process_all_files(chunk_size=chunk_size) # This was the call that was interrupted
            print(f"处理结果: {result}")

        elif choice == "4":
            show_results()

        elif choice == "5":
            show_examples()

        else:
            print("无效选择")

    except KeyboardInterrupt:
        print("\n\n操作已取消")
    except Exception as e:
        print(f"\n操作失败: {e}")

# 初始化说明
def initialize_colab():
    """初始化Colab环境"""
    # from part1 import initialize_environment

    print("🚀 初始化GMP问答对生成器...")
    print("📁 输入目录: ./processed/")
    print("📁 输出目录: ./jsons/")
    success = True
    if success:
        print("\n✓ 环境初始化完成")
        print("可以使用以下函数开始处理:")
        print("- list_available_files()  # 列出文件")
        print("- preview_file(filename)  # 预览文件")
        print("- process_single_file(filename)  # 处理单个文件")
        print("- process_all_files()  # 处理所有文件")
        print("- show_results()  # 显示结果")
        print("- show_examples()  # 显示示例")
    return success

if __name__ == "__main__":
    initialize_colab()
    main()

2025-11-27 11:05:36,244 - INFO - 在 ./processed 中找到 14 个文本文件


🚀 初始化GMP问答对生成器...
📁 输入目录: ./processed/
📁 输出目录: ./jsons/

✓ 环境初始化完成
可以使用以下函数开始处理:
- list_available_files()  # 列出文件
- preview_file(filename)  # 预览文件
- process_single_file(filename)  # 处理单个文件
- process_all_files()  # 处理所有文件
- show_results()  # 显示结果
- show_examples()  # 显示示例
GMP问答对生成器 - Colab交互模式
📁 在processed目录中找到 14 个文本文件:
   1. 20220701 PICS-GMPGDP 监管环境中数据管理和可靠性的良好实践_cleaned.txt (267,681 bytes)
   2. 已上市中药药学变更研究技术指导原则（试行）》_cleaned.txt (38,360 bytes)
   3. 2023editionGMPGuideOralSolidDosagetxt_cleaned.txt (654,271 bytes)
   4. DrugGMPGuideSterileProducts_Part2_2023edition_cleaned.txt (741,351 bytes)
   5. 2023editionGMPGuideAPItxt_cleaned.txt (1,234,771 bytes)
   6. GMPGuide2023edition_QualityManagementSystem_W_cleaned.txt (860,452 bytes)
   7. 202501清洁验证技术指南_cleaned.txt     (101,900 bytes)
   8. pe-009-16-gmp-guide-annexes_cleaned.txt (576,142 bytes)
   9. 已上市化学药品药学变更研究技术指导原则（试行）_cleaned.txt (63,865 bytes)
  10. 2023editionGMPGuideSterileProductsPart1txt_cleaned.txt (1,524,716 bytes)
  1

2025-11-27 11:06:06,623 - INFO - ✓ Google Colab AI客户端已初始化
2025-11-27 11:06:06,625 - INFO - 在 ./processed 中找到 14 个文本文件
2025-11-27 11:06:06,627 - INFO - 开始处理 14 个文件...
2025-11-27 11:06:06,628 - INFO - 开始处理文件: 20220701 PICS-GMPGDP 监管环境中数据管理和可靠性的良好实践_cleaned.txt


🚀 开始处理 14 个文件...

2025-11-27 11:06:06,866 - INFO - 文本质量评分: 30/100
2025-11-27 11:06:06,874 - INFO - 文档类型: GMP
2025-11-27 11:06:06,878 - INFO - 处理后的文本已保存: processed/20220701 PICS-GMPGDP 监管环境中数据管理和可靠性的良好实践_cleaned_cleaned.txt
2025-11-27 11:06:06,991 - INFO - 文本分块完成，共 1251 个块，平均长度: 354 字符
2025-11-27 11:06:15,784 - ERROR - Colab AI调用失败: object str can't be used in 'await' expression


...

📝 20220701 PICS-GMPGDP 监管环境中数据管理和可靠性的良好实践_cleaned.txt [░░░░░░░░░░░░░░░] 1% (7/1251块) 总进度: 0.0% Q&A: 0

2025-11-27 11:07:16,341 - ERROR - Colab AI调用失败: object str can't be used in 'await' expression


📝 20220701 PICS-GMPGDP 监管环境中数据管理和可靠性的良好实践_cleaned.txt [░░░░░░░░░░░░░░░] 1% (8/1251块) 总进度: 0.0% Q&A: 0

2025-11-27 11:07:24,036 - ERROR - Colab AI调用失败: object str can't be used in 'await' expression


📝 20220701 PICS-GMPGDP 监管环境中数据管理和可靠性的良好实践_cleaned.txt [░░░░░░░░░░░░░░░] 1% (9/1251块) 总进度: 0.0% Q&A: 0

操作已取消


In [ ]:
process_all_files(chunk_size=400, resume=True)

2025-11-27 11:11:07,054 - INFO - ✓ Google Colab AI客户端已初始化
2025-11-27 11:11:07,059 - INFO - 在 ./processed 中找到 15 个文本文件
2025-11-27 11:11:07,060 - INFO - 开始处理 15 个文件...
2025-11-27 11:11:07,063 - INFO - 开始处理文件: 20220701 PICS-GMPGDP 监管环境中数据管理和可靠性的良好实践_cleaned.txt


🚀 开始处理 15 个文件...

2025-11-27 11:11:07,289 - INFO - 文本质量评分: 30/100
2025-11-27 11:11:07,296 - INFO - 文档类型: GMP
2025-11-27 11:11:07,300 - INFO - 处理后的文本已保存: processed/20220701 PICS-GMPGDP 监管环境中数据管理和可靠性的良好实践_cleaned_cleaned.txt
2025-11-27 11:11:07,398 - INFO - 文本分块完成，共 1251 个块，平均长度: 354 字符


📝 20220701 PICS-GMPGDP 监管环境中数据管理和可靠性的良好实践_cleaned.txt [▓▓▓░░░░░░░░░░░░] 26% (321/1251块) 总进度: 0.0% Q&A: 0

2025-11-27 11:51:24,052 - ERROR - Colab AI调用失败: Error code: 429 - {'message': 'Insufficient quota available to perform this operation. Try again later.', 'type': 'invalid_request_error'}


...

📝 20220701 PICS-GMPGDP 监管环境中数据管理和可靠性的良好实践_cleaned.txt [▓▓▓▓░░░░░░░░░░░] 27% (339/1251块) 总进度: 0.0% Q&A: 0

2025-11-27 11:52:11,825 - ERROR - Colab AI调用失败: Error code: 429 - {'message': 'Insufficient quota available to perform this operation. Try again later.', 'type': 'invalid_request_error'}


📝 20220701 PICS-GMPGDP 监管环境中数据管理和可靠性的良好实践_cleaned.txt [▓▓▓▓░░░░░░░░░░░] 27% (340/1251块) 总进度: 0.0% Q&A: 0

2025-11-27 11:52:14,614 - ERROR - Colab AI调用失败: Error code: 429 - {'message': 'Insufficient quota available to perform this operation. Try again later.', 'type': 'invalid_request_error'}


📝 20220701 PICS-GMPGDP 监管环境中数据管理和可靠性的良好实践_cleaned.txt [▓▓▓▓░░░░░░░░░░░] 27% (341/1251块) 总进度: 0.0% Q&A: 0